In [1]:
from seleniumwire import webdriver
from selenium.webdriver.support.ui import Select

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

import time
import json


# data manipulate library
import pandas as pd

import csv

import stopit


In [2]:
import os
os.chdir('/Users/alisdghnia/Desktop/')

In [3]:
# Write the header
fieldnames = ['coin_name','link']

with open('result_2.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()

In [4]:
def WriteResult(result):
    # Now write to csv file
    fieldnames = ['coin_name','link']

    with open('result_2.csv', 'a', encoding='UTF8', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writerow(result)

In [5]:
def ExpandTo100Page():
    # make sure we have all the 100 pages
    dropdown_select = Select(driver.find_element(By.XPATH, '//select') )
    dropdown_select.select_by_value('100')

In [6]:
# Go to next page
def NextPage():
    next_button = driver.find_element(By.XPATH, '//a[@class="paginate_button next"]')
    next_button.click()


In [7]:
# Go to middle page and get pdf
def GoMiddlePage(middle_page_link):
    pdf_link = ''

    # chặn timeout
    with stopit.ThreadingTimeout(60) as context_manager:

        # clear request
        del driver.requests

        # open another tab
        driver.execute_script("window.open('');")
        driver.switch_to.window(driver.window_handles[1])
        driver.get(middle_page_link)


        try:
            
            # wait until see the pdf content
            driver.wait_for_request('/wp-content/uploads/', timeout=30)

    
            # sample code we want to run...
            for request in driver.requests:
                if  (request.response):
                    if ('.pdf' in str(request.url)):
                        pdf_link = str(request.url)
                        break
                
            # When we found the link, time to go back to orginal tab
            driver.close()
            driver.switch_to.window(driver.window_handles[0])

        except:
            driver.close()
            driver.switch_to.window(driver.window_handles[0])

    if context_manager.state == context_manager.EXECUTED:
        print("COMPLETE...")
        return pdf_link   


    # Did code timeout?
    elif context_manager.state == context_manager.TIMED_OUT:
        print("DID NOT FINISH...")
        return pdf_link
            




In [8]:
def FirstPage():
    # get the whole list
    whole_100_coins = driver.find_elements(By.XPATH, '//tbody//tr')


    for coin in whole_100_coins:
        result_dict = {}

        # get the coin name
        coin_name = coin.find_element(By.XPATH,'.//td[@class="column-1"]')
        coin_name = coin_name.text

        # get link to middle page
        middle_page_link = coin.find_element(By.XPATH,'.//td[@class="column-2"]//a')
        middle_page_link = middle_page_link.get_attribute('href')
        with stopit.ThreadingTimeout(60) as context_manager:
            pdf_link = GoMiddlePage(middle_page_link=middle_page_link)
            
        if context_manager.state == context_manager.EXECUTED:

            # okkay now write to csv file
            result_dict['coin_name'] = (coin_name)
            result_dict['link'] = (pdf_link)
            
            print(coin_name + ": " + pdf_link )
        
            WriteResult(result=result_dict)

In [9]:
driver = webdriver.Chrome()

driver.get('https://www.allcryptowhitepapers.com/whitepaper-overview/')
time.sleep(10)

for i in range(30):
    # make it expand to 100 pages
    time.sleep(5)

    ExpandTo100Page()

    FirstPage()
    
    NextPage()


COMPLETE...
Bitcoin: https://www.allcryptowhitepapers.com/wp-content/uploads/2018/05/bitcoin-whitepaper.pdf
COMPLETE...
Ethereum: https://www.allcryptowhitepapers.com/wp-content/uploads/2018/05/Ethereum-Whitepaper.pdf
COMPLETE...
Ripple - XRP: https://www.allcryptowhitepapers.com/wp-content/uploads/2018/05/ripple-consensus-whitepaper.pdf
COMPLETE...
Bitcoin Cash: https://www.allcryptowhitepapers.com/wp-content/uploads/2018/05/bitcoin-cash-whitepaper.pdf
COMPLETE...
Litecoin: 
COMPLETE...
Tether: https://www.allcryptowhitepapers.com/wp-content/uploads/2018/05/TetherWhitePaper.pdf
COMPLETE...
EOS: https://www.allcryptowhitepapers.com/wp-content/uploads/2018/05/EOS-.pdf
COMPLETE...
Binance Coin: 
COMPLETE...
Bitcoin Cash SV: https://www.allcryptowhitepapers.com/wp-content/uploads/2018/11/Bitcoin-SV-Whitepaper.pdf
COMPLETE...
Monero: https://www.allcryptowhitepapers.com/wp-content/uploads/2018/05/monero-whitepaper.pdf
COMPLETE...
Cardano: https://www.allcryptowhitepapers.com/wp-content/upl

ElementClickInterceptedException: Message: element click intercepted: Element <a class="paginate_button next" aria-controls="tablepress-34" data-dt-idx="1" tabindex="0" id="tablepress-34_next">...</a> is not clickable at point (809, 1091). Other element would receive the click: <iframe id="aswift_2" name="aswift_2" style="width: 1005px !important; height: 124px !important; display: block; margin: 0px auto;" sandbox="allow-forms allow-popups allow-popups-to-escape-sandbox allow-same-origin allow-scripts allow-top-navigation-by-user-activation" width="1005" height="124" frameborder="0" marginwidth="0" marginheight="0" vspace="0" hspace="0" allowtransparency="true" scrolling="no" src="https://googleads.g.doubleclick.net/pagead/html/r20221101/r20110914/zrt_lookup.html?fsb=1#RS-1-&amp;adk=1812271801&amp;client=ca-pub-8951348962689436&amp;fa=1&amp;ifi=3&amp;uci=a!3&amp;btvi=1&amp;xpc=WDbBgr8MVt&amp;p=https%3A//www.allcryptowhitepapers.com" data-google-container-id="a!3" data-google-query-id=